In [ ]:
import sys
import cv2
import glob
import os
import math
import csv
import numpy as np
from pathlib import Path
from collections import deque

sys.path.append(str(Path.cwd().parent))
from utils.file_dialog_utils import pick_video_cv2, pick_folder

# --- PART 1: LOADING & PREPARATION ---

def load_templates_with_names(folder_path):
    """
    Loads templates and keeps their filenames.
    Returns: list of (filename, image_bgr)
    """
    paths = sorted(glob.glob(os.path.join(folder_path, "*.png")))
    template_data = []
    for p in paths:
        img = cv2.imread(p)
        if img is not None:
            filename = os.path.basename(p)
            template_data.append((filename, img))
    return template_data

def preprocess_templates_for_video(template_data):
    """
    Converts BGR templates to Grayscale ONCE to speed up video processing.
    Returns: list of (name, gray_image, width, height)
    """
    processed = []
    for name, img_bgr in template_data:
        gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
        processed.append((name, gray, w, h))
    return processed

# --- PART 2: MATCHING LOGIC ---

def find_best_match_fast(target_gray, gray_templates):
    """
    Optimized matcher for video. 
    Accepts PRE-CONVERTED gray templates to save time.
    """
    best_match = {
        "score": -1.0,
        "location": None,
        "width": 0,
        "height": 0,
        "name": ""
    }

    for name, tmpl_gray, w, h in gray_templates:
        res = cv2.matchTemplate(target_gray, tmpl_gray, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        if max_val > best_match["score"]:
            best_match["score"] = max_val
            best_match["location"] = max_loc
            best_match["width"] = w
            best_match["height"] = h
            best_match["name"] = name

    return best_match

In [ ]:
# --- PART 3: VIDEO PROCESSING ---

def process_video(video_path, template_folder, output_path="matched_output.mp4"):
    # 1. Setup
    print(f"Opening video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties for the Output Writer
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Setup Video Writer (Try MP4, fallback to AVI if needed)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 2. Load and Preprocess Templates
    print("Loading and optimizing templates...")
    raw_templates = load_templates_with_names(template_folder)
    gray_templates = preprocess_templates_for_video(raw_templates)
    
    if not gray_templates:
        print("Error: No templates found.")
        return

    print(f"Starting processing on {total_frames} frames with {len(gray_templates)} templates.")
    print("-" * 50)

    # 3. Main Loop
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break # End of video

        # Convert frame to gray
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Find best match
        result = find_best_match_fast(frame_gray, gray_templates)

        # Draw result if found
        if result['location']:
            top_left = result['location']
            bottom_right = (top_left[0] + result['width'], top_left[1] + result['height'])
            
            # Green Box
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
            
            # Text Label (Score)
            label = f"{result['score']:.2f}"
            cv2.putText(frame, label, (top_left[0], top_left[1]-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Write frame to output video
        writer.write(frame)

        # Progress Update (Every 10 frames to avoid spamming)
        frame_idx += 1
        if frame_idx % 10 == 0:
            print(f"Processing frame {frame_idx}/{total_frames}...")

    # 4. Cleanup
    cap.release()
    writer.release()
    print("-" * 50)
    print(f"Done! Video saved to: {output_path}")

In [ ]:
def fit_circle_least_squares(points_xy: np.ndarray):
    """
    Simple least-squares circle fit.
    points_xy: Nx2 array
    Returns: (cx, cy, r)
    """
    x = points_xy[:, 0]
    y = points_xy[:, 1]
    A = np.c_[2*x, 2*y, np.ones_like(x)]
    b = x**2 + y**2
    sol, *_ = np.linalg.lstsq(A, b, rcond=None)
    cx, cy, c = sol
    r = math.sqrt(max(0.0, c + cx**2 + cy**2))
    return float(cx), float(cy), float(r)


def fit_ellipse(points_xy):
    """
    points_xy: list of (x, y) or Nx2 numpy array
    returns: (center(x,y), axes(rx, ry), angle_deg)
    """
    pts = np.array(points_xy, dtype=np.float32).reshape(-1, 1, 2)
    (cx, cy), (MA, ma), angle = cv2.fitEllipse(pts)
    # MA, ma are full axis lengths -> convert to radii
    rx, ry = MA / 2.0, ma / 2.0
    return (float(cx), float(cy)), (float(rx), float(ry)), float(angle)


def calculate_distance(p1, p2):
    """Calculates Euclidean distance between two (x, y) points."""
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


def process_video_with_distance_stabilization(video_path, template_folder, output_csv_path="zero_points.csv", overlay_image_path="circle_overlay_first_frame.png", max_distance=50, min_score=0.6):
    # 1) Setup Video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Read first frame now (for later overlay), then reset back to start
    ok, first_frame = cap.read()
    if not ok:
        print("Error: Could not read first frame.")
        cap.release()
        return
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # 2) Setup Templates
    print("Loading templates...")
    templates_with_names = load_templates_with_names(template_folder)
    gray_templates = preprocess_templates_for_video(templates_with_names)
    print(f"Loaded {len(gray_templates)} templates.")

    detection_history = deque(maxlen=3)
    last_valid_location = None  # (x, y) top-left of last accepted match

    # We'll store only stable detections here
    stable_rows = []

    print(f"Starting Distance-Based Tracking (CSV output)...")
    print(f"Distance Threshold: {max_distance} px | min_score: {min_score}")
    print("-" * 50)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Find best match this frame
        current_match = find_best_match_fast(frame_gray, gray_templates)
        current_loc = current_match["location"]
        current_score = current_match["score"]

        is_good_frame = False

        if current_loc is None:
            is_good_frame = False

        elif last_valid_location is None:
            # Lock-on using score
            if current_score >= min_score:
                is_good_frame = True
                last_valid_location = current_loc
            else:
                is_good_frame = False

        else:
            # Check if it jumped too far
            dist = calculate_distance(current_loc, last_valid_location)
            if dist <= max_distance and current_score >= min_score:
                is_good_frame = True
                last_valid_location = current_loc
            else:
                is_good_frame = False
                # do NOT update last_valid_location

        # Update queue
        detection_history.append(1 if is_good_frame else 0)

        # Only accept when stable for 3 consecutive frames
        if len(detection_history) == 3 and sum(detection_history) == 3:
            top_left = current_match["location"]
            w, h = current_match["width"], current_match["height"]

            # Convert to center coordinate
            cx = top_left[0] + w / 2.0
            cy = top_left[1] + h / 2.0

            stable_rows.append({
                "frame_idx": frame_idx,
                "center_x": float(cx),
                "center_y": float(cy),
                "score": float(current_score),
                "template": current_match["name"],
            })

        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"Processed {frame_idx}/{total_frames} frames...")

    cap.release()

    print("-" * 50)
    print(f"Stable detections: {len(stable_rows)}")

    # 3) Save CSV
    with open(output_csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["frame_idx", "center_x", "center_y", "score", "template"])
        writer.writeheader()
        writer.writerows(stable_rows)

    print(f"Saved CSV: {output_csv_path}")

    # 4) Fit ellipse + overlay on first frame
    if len(stable_rows) < 5:
        print("Not enough stable points to fit an ellipse (need at least 5). Falling back to circle.")
        pts = np.array([[r["center_x"], r["center_y"]] for r in stable_rows], dtype=np.float64)
        cx, cy, rad = fit_circle_least_squares(pts)

        overlay = first_frame.copy()
        cv2.circle(overlay, (int(cx), int(cy)), int(rad), (0, 0, 255), 3)
        cv2.circle(overlay, (int(cx), int(cy)), 4, (0, 255, 0), -1)
        cv2.imwrite(overlay_image_path, overlay)
        print(f"Saved overlay image (circle fallback): {overlay_image_path}")
        print(f"Circle fit -> center=({cx:.1f}, {cy:.1f}), radius={rad:.1f}px")
        return

    # Fit ellipse
    points_xy = [(r["center_x"], r["center_y"]) for r in stable_rows]
    (center, axes, angle) = fit_ellipse(points_xy)
    (cx, cy) = center
    (rx, ry) = axes

    overlay = first_frame.copy()
    cv2.ellipse(
        overlay,
        (int(cx), int(cy)),
        (int(rx), int(ry)),
        angle,
        0, 360,
        (0, 0, 255),
        3
    )
    cv2.circle(overlay, (int(cx), int(cy)), 4, (0, 255, 0), -1)

    cv2.imwrite(overlay_image_path, overlay)
    print(f"Saved overlay image (ellipse): {overlay_image_path}")
    print(f"Ellipse fit -> center=({cx:.1f}, {cy:.1f}), axes=({rx:.1f}, {ry:.1f}), angle={angle:.1f}°")

In [ ]:
# --- MAIN EXECUTION ---

# Settings
_, input_video_path = pick_video_cv2(title="Select Input Video")
templates_dir = pick_folder(title="Select Template Folder")

# Run
process_video_with_distance_stabilization(input_video_path, templates_dir)